In [17]:
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output
import pandas as pd
import json
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import numpy as np

In [18]:
df = pd.read_csv('data_model_dashboard.csv', sep='\t')
print (df.columns)

clients=df["SK_ID_CURR"]
threshold=0.14

Index(['index', 'SK_ID_CURR', 'TARGET', 'CODE_GENDER', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY',
       ...
       'CC_NAME_CONTRACT_STATUS_Signed_MEAN',
       'CC_NAME_CONTRACT_STATUS_Signed_SUM',
       'CC_NAME_CONTRACT_STATUS_Signed_VAR', 'CC_NAME_CONTRACT_STATUS_nan_MIN',
       'CC_NAME_CONTRACT_STATUS_nan_MAX', 'CC_NAME_CONTRACT_STATUS_nan_MEAN',
       'CC_NAME_CONTRACT_STATUS_nan_SUM', 'CC_NAME_CONTRACT_STATUS_nan_VAR',
       'CC_COUNT', 'TARGET_adj'],
      dtype='object', length=799)


In [19]:
fi=pd.read_csv('feature_importance.csv',sep='\t')
fi=fi.sort_values(by="importance", ascending=False)
print (fi)
available_indicators=fi["feature"]

                             feature  importance
5                    YEARS_BUILD_AVG    0.053152
12                  YEARS_BUILD_MODE    0.045301
20                NONLIVINGAREA_MEDI    0.023079
3                       EXT_SOURCE_3    0.019735
2                       EXT_SOURCE_2    0.015368
16                NONLIVINGAREA_MODE    0.014646
18                  YEARS_BUILD_MEDI    0.012881
10                 NONLIVINGAREA_AVG    0.007596
23            EMERGENCYSTATE_MODE_No    0.007216
38      CC_AMT_TOTAL_RECEIVABLE_MEAN    0.006643
13                   COMMONAREA_MODE    0.006358
35          INSTAL_PAYMENT_DIFF_MEAN    0.006157
4        YEARS_BEGINEXPLUATATION_AVG    0.006080
31   ACTIVE_MONTHS_BALANCE_SIZE_MEAN    0.005814
22     HOUSETYPE_MODE_block of flats    0.005568
14                    FLOORSMIN_MODE    0.005119
36           INSTAL_PAYMENT_DIFF_SUM    0.005112
28    BURO_CREDIT_ACTIVE_Closed_MEAN    0.005067
30           BURO_STATUS_C_MEAN_MEAN    0.004802
34          INSTAL_P

In [20]:
#https://www.datacamp.com/community/tutorials/learn-build-dash-python



In [21]:
'''y_value=int(df[df['SK_ID_CURR'] ==115304]['INSTAL_PAYMENT_PERC_MAX'])

y_target=int(df[df['SK_ID_CURR'] == 115304]['TARGET_adj'])

y_min=df['INSTAL_PAYMENT_PERC_MAX'].min()
y_max=df['INSTAL_PAYMENT_PERC_MAX'].max()

figure_curves=ff.create_distplot([df[df['TARGET_adj'] == 0]['INSTAL_PAYMENT_PERC_MAX'],df[df['TARGET_adj'] == 1]['INSTAL_PAYMENT_PERC_MAX']],['Score=0','Score=1'], show_hist=False,show_rug=False)
figure_curves.add_trace(
    go.Scatter(
        x=[float(df[df['SK_ID_CURR'] == 115304]['INSTAL_PAYMENT_PERC_MAX']),float(df[df['SK_ID_CURR'] == 115304]['INSTAL_PAYMENT_PERC_MAX'])],
        y=[figure_curves['data'][y_target]['y'].min(),figure_curves['data'][y_target]['y'].max()],
        mode="lines",
        line=go.scatter.Line(color="gray"),
        name='Client',
        showlegend=True)
)'''

print (df[df['INSTAL_PAYMENT_PERC_MAX']>10]['SK_ID_CURR'])


0      167820.0
4      387235.0
9      295015.0
11     332382.0
14     165092.0
         ...   
670    271682.0
672    212033.0
686    185075.0
705    307648.0
712    389091.0
Name: SK_ID_CURR, Length: 62, dtype: float64


In [51]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

app.layout = html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in clients],
                value=115304
            , style={
                'borderBottom': 'thin lightgrey solid',
                'backgroundColor': 'rgb(250, 250, 250)',
                'padding': '10px 5px',
                'text-align': 'center',
            }),
        ],className="six columns"),
        html.Div([
        dcc.Dropdown(
            id='crossfilter-var-column',
            options=[{'label': i, 'value': i} for i in available_indicators],
            value='INSTAL_PAYMENT_PERC_MAX'
        , style={
            'borderBottom': 'thin lightgrey solid',
            'backgroundColor': 'rgb(250, 250, 250)',
            'padding': '10px 5px',
            'text-align': 'center',
            })
         ],className="six columns"),
    ],className="row"),
    html.Div([
        dcc.Textarea(
            id='textarea-example',
            value='Score = ',
            style={
                'width': '100%',
                'height': '100%',
                'resize':'none',
                'border':'2px solid black',
                'text-align':'center',
                'font-size':'40px'}
        ),
    ]),
    html.Div([
        html.Div([
            dcc.Graph(
                id='feature_importance',
                figure={
                    'data': [
                        {
                        'x': fi['importance'],
                        'y': fi['feature'],
                        'type': 'bar',
                        'orientation':'h'
                        }
                    ],
                    'layout': go.Layout(
                        xaxis={'title': 'Importance'},
                        yaxis={'title': 'Feature'},
                        hovermode='closest',
                        width=100,
                    )
                }    
            )
        ],className="six columns"),
        html.Div([
            dcc.Graph(
                id='graph_three_curves',
                figure=figure_curves       
            )
         ],className="six columns"),
    ],className="row"),
    html.Div([
        html.Div([
            dcc.Dropdown(
                id='crossfilter-var1-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='INSTAL_PAYMENT_PERC_MAX'
            )
        ],className="six columns"),
        html.Div([
            dcc.Dropdown(
                id='crossfilter-var2-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='INSTAL_PAYMENT_PERC_MAX'
            )
        ],className="six columns")
    ],className="row"),
    dcc.Graph(
        id='graph_two_variables',
        figure={
            'data': [
                {
                    'x': df[df['TARGET_adj'] == 0]['TARGET'],
                    'y': df[df['TARGET_adj'] == 0]['INSTAL_PAYMENT_PERC_MAX'],
                    'name': 'Score = 0',
                    'mode': 'markers',
                    'marker': {'size': 12,'color':'blue','opacity':0.5}
                },
                {
                    'x': df[df['TARGET_adj'] == 1]['TARGET'],
                    'y': df[df['TARGET_adj'] == 1]['INSTAL_PAYMENT_PERC_MAX'],
                    'name': 'Score = 1',
                    'mode': 'markers',
                    'marker': {'size': 12,'color':'orange','opacity':0.5}
                },
                {
                    'x': df[df['SK_ID_CURR'] == 115304]['TARGET'],
                    'y': df[df['SK_ID_CURR'] == 115304]['INSTAL_PAYMENT_PERC_MAX'],
                    'name': 'ID Client',
                    'mode': 'markers',
                    'marker': {'size': 12,'color':'Chartreuse','opacity':1}
                }
            ],
            'layout': go.Layout(
                xaxis={'title': 'TARGET'},
                yaxis={'title': 'INSTAL_PAYMENT_PERC_MAX'},
                margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
                hovermode='closest'
            )
        }    
    )
])

@app.callback(dash.dependencies.Output('textarea-example', 'value'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value')])
def update_output(value):
    score=(df.loc[df["SK_ID_CURR"]==int(value),'TARGET_adj']).values
    text=""
    if score == 0 :
        text = "Crédit accepté"
    elif score == 1 :
        text = "Crédit refusé"
    return ('Score = {}\n{}'.format(score[0],text))


@app.callback(
    dash.dependencies.Output('graph_two_variables', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-var1-column', 'value'),
     dash.dependencies.Input('crossfilter-var2-column', 'value'),])
def update_timeseries(xaxis_column_name,var1_column_name,var2_column_name):
    return {
            'data': [
                {
                    'x': df[df['TARGET_adj'] == 0][var1_column_name],
                    'y': df[df['TARGET_adj'] == 0][var2_column_name],
                    'name': 'Score = 0',
                    'mode': 'markers',
                    'marker': {'size': 12,'color':'blue','opacity':0.5}
                },
                {
                    'x': df[df['TARGET_adj'] == 1][var1_column_name],
                    'y': df[df['TARGET_adj'] == 1][var2_column_name],
                    'name': 'Score = 1',
                    'mode': 'markers',
                    'marker': {'size': 12,'color':'orange','opacity':0.5}
                },
                {
                    'x': df[df['SK_ID_CURR'] == xaxis_column_name][var1_column_name],
                    'y': df[df['SK_ID_CURR'] == xaxis_column_name][var2_column_name],
                    'name': 'ID Client',
                    'mode': 'markers',
                    'marker': {'size': 12,'color':'Chartreuse','opacity':1}
                }
            ],
            'layout': go.Layout(
                xaxis={'title': var1_column_name},
                yaxis={'title': var2_column_name},
                margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
                hovermode='closest'
            )
        }

@app.callback(
    dash.dependencies.Output('graph_three_curves', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-var-column','value'),
     dash.dependencies.Input('feature_importance', 'clickData')])
def update_timeseries(xaxis_column_name,var_column_name,clickData):        
    y_value=int(df[df['SK_ID_CURR'] ==xaxis_column_name][var_column_name])
    y_target=int(df[df['SK_ID_CURR'] == xaxis_column_name]['TARGET_adj'])
    y_min=df['INSTAL_PAYMENT_PERC_MAX'].min()
    y_max=df['INSTAL_PAYMENT_PERC_MAX'].max()

    figure_curves=ff.create_distplot([df[df['TARGET_adj'] == 0][var_column_name],df[df['TARGET_adj'] == 1][var_column_name]],['Score=0','Score=1'], show_hist=False,show_rug=False)
    figure_curves.add_trace(
        go.Scatter(
            x=[float(df[df['SK_ID_CURR'] == xaxis_column_name][var_column_name]),float(df[df['SK_ID_CURR'] == xaxis_column_name][var_column_name])],
            y=[figure_curves['data'][y_target]['y'].min(),figure_curves['data'][y_target]['y'].max()],
            mode="lines",
            line=go.scatter.Line(color="gray"),
            name='Client',
            showlegend=True
        )
    )
    return figure_curves


def update_trace(xaxis_column_name,clickData):
    y_target=int(df[df['SK_ID_CURR'] == xaxis_column_name]['TARGET_adj'])
    return fig.update_traces(
        x=[float(df[df['SK_ID_CURR'] == xaxis_column_name][clickData['points'][0]['label']]),float(df[df['SK_ID_CURR'] == xaxis_column_name][clickData['points'][0]['label']])],
        y=[figure_curves['data'][y_target]['y'].min(),figure_curves['data'][y_target]['y'].max()],
        selector=dict(mode="lines"),overwrite=True)

In [52]:
app.run_server()


Dash app running on http://127.0.0.1:8050/


In [24]:
# Pour le déploiement avec heroku: https://dash.plotly.com/deployment